## Azure AI Search Index: Step-by-step Guide

In this tutorial, we’ll focus on creating an index in Azure AI Search and then integrate it as a function tool within a simple agent built with Microsoft Agent Framework. This setup will allow our agent to retrieve relevant information from the search index and respond intelligently in a chat scenario.


#### Step 1: Create a Resource Group

First, we need to create a Resource Group (RG) in Azure. This RG will serve as the container for all the services we’ll use Azure AI Search Service, Storage Account and Azure OpenAI Service.

Keeping these resources in one RG makes management and cleanup easier.

1.**Sign in to the Azure Portal**
    
Go to https://portal.azure.com and log in with your Azure account.



2. **Navigate to Resource Groups**

    - In the left-hand menu, click Resource groups.
    - Then click + Create at the top.

3. **Fill in the Resource Group details**

    - Subscription: Select the subscription you want to use.
    - Resource Group Name: Choose a clear name, e.g., rg-ai-services.
    - Region: Pick the region closest to your users or where you want to deploy services (e.g., West Europe).


4. **Review and Create**

    - Click Review + Create, then Create.
    - Wait for the deployment to finish.

#### Step 2: Create an Azure AI Search service

We will be using Azure AI search in this exercise, so we need to set this up. Navigate to the same resource group where you created your AI project and create a new Azure AI Search service resource.

1. It is recommended to create the search service in the same region as your AI project
2. Select 'Standard' tier, which is required for these labs

![img1](images/image.png)

3. Navigate to your search service once it is created. Under Settings > Keys, ensure 'Both' is selected for API access control. Click save.

![img2](images/image1.png)

4. Navigate to Settings > Identity > turn on system-assigned identity and click 'Save'.

![img3](images/image2.png)

#### Step 2: Create an Azure OpenAI service and deploy an embedding model

Next, we need to create an Azure OpenAI service and deploy a text embedding model there. We will use this text embedding model later to vectorize our documents.

1. Create a new Azure OpenAI service resource in the same resource group we just creaed.
2. In your newly created Azure OpenAI service, click Go to Azure AI Foundry portal.
3. Under 'Shared resources' section, click on the 'Deployments'.
4. Click on 'Deploy model' and then 'Deploy base model'.
5. Select 'text-embedding-3-large' and deploy it. This will be used later to create the vector embeddings for our documents.

#### Step 3: Create a Storage Account

We need to create a storage account and within it, storage container, which will store our documents. This is where our Azure AI Search service will retrieve documents from. Create a new Storage Account resource in the same RG as previous resources:

![Sample Photo](images/create-SA.jpg)

#### Step 7: Required permissions for the lab


##### Managed-identity permissions:

The next steps are needed for the AI search resource to be able to vectorize the input documents:

1. Navigate to the created storage account's **Access Control (IAM)** section. Select 'Add'->'Add role assignment'. **Assign the 'Storage Blob Data Reader role' to the search service identity**, as shown here:

![Sample Photo](images/blob-roleassign.jpg)

2. Navigate to your **Azure OpenAI resource** that was created above. **Assign the 'Cognitive Services OpenAI User' role to the search service identity**. To do this, select Access control > Add+ Role assignment > select Cognitive Services OpenAI User role > next select the managed identity of your search service > create role assignment.

##### User permissions:

**If you have contributor or higher permissions on a resource group, you inherit these user permissions and DO NOT need to add the below role assignments**, you can proceed to the next step. Otherwise, if you are a user, check to ensure the below role assignments are added for you.

TIP: if you are working on project teams and need to assign permissions to multiple users, you can add these permissions faster by creating a Entra ID security group (so you do not have to add users one-by-one) and by running Azure CLI scripts in Azure Cloud Shell.

1. At a minimum, users running these labs should have a **Reader role assigned on the Azure Subscription level**. In the Azure Portal, navigate to the Azure Subscription in which your resource group lives. On the left pane, select Access control > Role assignments. Filter 'Scope' to 'This resource'. Review user account permissions. If this is not asssigned, add this role assignment to the user.

2. **Each user needs to be granted the 'Cognitive Services OpenAI Contributor' role on the Azure AI Foundry resource**. To do this, in the Azure Portal, navigate to the Azure AI Foundry resource, select Access control > Add+ Role assignment > select Cognitive Services OpenAI Contributor role > next select the user(s) > create role assignment.

3. **Each user needs to be granted the 'Azure AI Project Manager' role on the Azure AI Foundry resource**. To do this, in the Azure Portal, navigate to the Azure AI Foundry resource, select Access control > Add+ Role assignment > select Azure AI Project Manager role > next select the user(s) > create role assignment. 

4. **Each user needs to be granted the 'Azure AI Project Manager' role in their resprective Azure AI Foundry project**. To do this, in the Azure Portal, navigate to the appropriate Foundry project, select Access control > Add+ Role assignment > select Azure AI Project Manager role > next select the user(s) > create role assignment. Repeat this process, as necessary, for all of the Foundry projects within your Foundry Resource.

5. **Each user needs to be granted the 'Storage Blob Data Contributor' role on the Storage Account**. To do this, add another role assignment > select Storage Blob Data Contributor role > next select the user(s) > create role assignment.

6. **Each user needs to be granted the 'Search Service Contributor' role on the Azure AI Search resource**. To do this, add another role assignment > select Search Service Contributor role > next select the user(s) > create role assignment.

Perfect! Now that we are done, you should have a Search service, a storage account and an Azure OpenAI model deployed in a resource group on Azure.

### Part 2: Create the Azure AI Search Index

We'll start the lab by creating an Azure AI Search index in our search service, which will contain vectorized representations of our documents. The steps shown below to create the AI Search index are from the official [Microsoft Documentation](https://learn.microsoft.com/en-us/azure/search/search-get-started-portal-import-vectors?tabs=sample-data-storage%2Cmodel-aoai%2Cconnect-data-storage)

#### Step 1
Inspect the documents in the 'data/index1' folder in the root of the '03-rag' folder. These will be stored in our Azure AI Search index.

#### Step 2
Next, we'll upload the documents to Azure Blob Storage, which Azure AI search connects to.
1. Navigate to the Storage Account you've just created. 
2. Expand "Data Storage" in the side menu and click on "Containers". 
3. Create a new container named: [your-initials] + "infodocs" (Example: 'kkinfodocs'). If you are working in a project team, everyone will be working in the same Storage Account, so we want to make sure our Blob Storage Container names are unique!

    ![image3.png](images/image3.png)
4. Click on the new container and upload the documents in the 'data' folder.

    ![image4.png](images/image4.png)
    
*Note*: if you get a warning about not having the permissions to list the data using your user account, go to 'Access Control (IAM)' and assign the 'Storage Blob Data Owner' role to your user

#### Step 3
We need an embedding model in order to convert our documents into vectors that will be stored in Azure AI Search. We've already deployed it previosuly in our Azure OpenAI service.

#### Step 4
Now we're ready to vectorize our documents.
1. Go to your Azure AI Search Service
2. On the **Overview page**, select **Import data (new)**.
3. Select **Azure Blob Storage** as your data source.
4. On the next page, select **RAG**.
5. On the next page **configure your Azure blob storage** by specifying your subscription, storage account, and the container that contains your healthplan documents 
6. Make sure **Authenticate using managed idenity is checked** and the **Managed identity type** is set to **System-assigned**.

    ![Sample Photo](images/search-blobconnect.jpg)

5. On the **Vectorize your text** page, select **Azure OpenAI** for **Kind**, select your subscription, and select the name of your Azure OpenAI Service.
6. For the **Model deployment** select `text-embedding-3-large`. 
7. The **Authentication type** should be set to **System-assigned identity**. 
8. Select the box next to the acknowledgement.

    ![Sample Photo](images/vectorize-model.jpg)

9. You can hit **Next** for the next two pages. When you get to the **Review and create** page, set **Objects name prefix** to **[your-blob-container-name]-index** (Example: 'efwhealthplan-index'). Click **Create**. This will start the document indexing process which will vectorize your documents and create an index. NOTE: it may take a few minutes for you index to fully populate with your document chunks. Wait until the process is complete BEFORE continuing to part 2.

    ![Sample Photo](images/index-name.jpg)

### TODO - Overview of what happened in the background

What has happened in the background, what is the process of creating an index etc


The setup is done! Now you can proceed - go to the `1.2-rag_agent.ipynb` notebook to create the Search Agent.